In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import Adam
from numpy import load

In [2]:
'''
after having understood how we prepare the celeb10 data for the GAN
(check CelebA - Preparing the data)
we now implement it,

we'll first define the D

it'll take an input of (80,80,3) - image's pixel data array

over 5 conv + leakyrelu layers we'll get

80 -> 40 -> 20 -> 10 -> 5

then we'll add 1 more similar layer and pass the output to a flatten + dropout + dense

add to it adam and compile
'''

def make_D(in_shape = (80, 80, 3)):
    model = Sequential()

    model.add(Conv2D(128, (5,5), padding = 'same', input_shape = in_shape))
    model.add(LeakyReLU(alpha = 0.2))

    model.add(Conv2D(128, (5,5), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))

    model.add(Conv2D(128, (5,5), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))

    model.add(Conv2D(128, (5,5), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))

    model.add(Conv2D(128, (5,5), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation = 'sigmoid'))

    opt = Adam(learning_rate = 0.0002, beta_1 = 0.5)
    model.compile(loss = 'binary_crossentropy', optimizer = opt, meter = ['accuracy'])
    return model

In [3]:
'''
and now the generator
'''

def define_generator(latent_dim):
    model = Sequential()

    n_nodes = 128 * 5 * 5

    model.add(Dense(n_nodes, input_dim = latent_dim))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Reshape((5, 5, 128)))

    model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))

    model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))

    model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))

    model.add(Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same'))
    model.add(LeakyReLU(alpha = 0.2))

    model.add(Conv2D(3, (5,5), activation = 'tanh', padding = 'same'))

    return model

In [4]:
def make_GAN(gen, dis):
    dis.trainable = False

    gan = Sequential()

    gan.add(gen)
    gan.add(dis)

    opt = Adam(learning_rate = 0.0002, beta_1 = 0.5)

    gan.compile(loss = 'binary_crossentropy', optimizer = opt)

    return gan

In [5]:
def load_real_samples():
    data = load('data.npz')
    X = data['arr_0']

    X = X.astype('float32')

    X = (X - 127.5) / 127.5

    return X

In [13]:
from numpy.random import randint
from numpy import ones

def generate_real_samples(dataset, n_samples):
    i = randint(0, dataset.shape[0], n_samples)

    X = dataset[i]

    y = ones((n_samples, 1))
    return X, y

In [11]:
from numpy.random import randn

def generate_latent_points(latent_dim, n_samples):
    X = randn(latent_dim * n_samples)

    X = X.reshape(n_samples, latent_dim)
    return X

In [12]:
from numpy import zeros

def generate_fake_samples(gen, latent_dim, n_samples):
    X = generate_latent_points(latent_dim, n_samples)

    X_in = gen.predict(X, verbose = 'off')

    y = zeros((n_samples, 1))
    return X_in, y

In [ ]:
def train_gan(gan, gen, dis, dataset, latent_dim, n_epochs = 20, batch_size = 128):
    batch_p_epoch = dataset.shape[0] // batch_size
    half_batch = batch_size // 2

    for i in range(n_epochs):
        for j in range(batch_p_epoch):

            X_real, y_real = generate_real_samples(dataset, half_batch)

            d_lossR, _ = dis.train_on_batch(X_real, y_real)

            X_fake, y_fake = generate_fake_samples(gen, latent_dim, half_batch)

            d_lossF, _ = dis.train_on_batch(X_fake, y_fake)

            X_gan = generate_latent_points(latent_dim, batch_size)

            y_gan = ones((batch_size, 1))

            gan_loss = gan.train_on_batch(X_gan, y_gan)

            print(f'{i}/{n_epochs} - {j}/{batch_p_epoch} - D_lossR : {d_lossR:.2f} - D_lossF : {d_lossF:.2f} - GAN_loss : {gan_loss:.2f}')

            if (i+1) % 10 == 0:
                # need a view_performance() here

In [ ]:
''' In Sha Allah
    will continue with view_performance() tomorrow
'''